In [126]:
import io
import json
import os
import random
import subprocess
from contextlib import redirect_stdout

import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras.layers import Conv2D, Conv2DTranspose, Input, LeakyReLU
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.multi_gpu_utils import multi_gpu_model
from matplotlib.colors import hsv_to_rgb, rgb_to_hsv
from PIL import Image, ImageEnhance

from sklearn import svm
import tensorflow.io.gfile as gfile


import trainer.parse_file_list
from trainer.gpu_utils import ModelCkptMultiGPU, get_available_gpus
from trainer.model_ckpt_gc import ModelCheckpointGC
from trainer.plot_utils import plot_hist, plot_mvtec, plot_samples, savefig

# from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard
# from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Input
# from tensorflow.keras.models import Model, load_model

In [62]:
# set random seed for redproducible results
RAND = 42
np.random.seed(RAND)
tf.set_random_seed(RAND)
random.seed(RAND)

In [3]:
from types import SimpleNamespace

In [70]:
BUCKET = "gs://e4u-anomaly-detection"
args  = SimpleNamespace(use_case="spm",
                        datadir="gs://training-data-images/",
                       logdir="gs://e4u-anomaly-detection/logs/DEBUG_AD",
                       ckptdir="gs://e4u-anomaly-detection/ckpts/DEBUG_AD",
                       imgdir="gs://e4u-anomaly-detection/imgs/DEBUG_AD",
                       evaldir="gs://e4u-anomaly-detection/eval/DEBUG_AD",
                       epochs=50,
                       batch_size=128,
                       filters=[32,32,32,64,64,128,256,128,64],
                       ldim=256)

global USE_CASE
USE_CASE = args.use_case

In [71]:
def read_image_file_names(dir_path):
    """Returns a list of absolute file paths for relative dir input with all relevant file names."""
    return [os.path.join(dir_path, p) for p in gfile.ListDirectory(dir_path)]

In [72]:
def get_path_to_data(data_path, mode):
    return os.path.join(data_path, USE_CASE, mode)

In [120]:
def load_data(data_path):
    normal_path = os.path.join(data_path, USE_CASE, "train", "good")
    normal_file_names = read_image_file_names(normal_path)
    random.shuffle(normal_file_names)

    train_validation_split = (len(normal_file_names) // 10) * 8  # equals .8
    train_file_names, validation_file_names = normal_file_names[
        :train_validation_split], normal_file_names[train_validation_split:]

    test_path = os.path.join(data_path, USE_CASE, "test")

    normal_test_file_names = read_image_file_names(
        os.path.join(test_path, "good"))

    anomaly_test_file_names = anomaly_train_file_names = []
    # iterate of test list dir because anomalies are named dynamically
    for i, p in enumerate(filter(lambda x: x != "good", gfile.ListDirectory(test_path))):
        if i == 0:
            anomaly_test_file_names += read_image_file_names(
                os.path.join(test_path, p))
        else:
            anomaly_train_file_names += read_image_file_names(
                os.path.join(test_path, p))

    print("Splitting data:\n%4d Normal Train Samples\n%4d Normal Validation Samples\n%4d Normal Test Samples\n%4d Anomaly Train Samples\n%4d Anomaly Test Samples" % (
        len(train_file_names), len(validation_file_names), len(normal_test_file_names), len(anomaly_train_file_names), len(anomaly_test_file_names)))

    return train_file_names, validation_file_names, normal_test_file_names, anomaly_train_file_names, anomaly_test_file_names

In [ ]:
import tensorflow as tf
import tensorflow.io.gfile
import os
from urllib.parse import urlparse
import numpy as np
import base64

DATA_SET_FRAC = 50 

In [ ]:
def load_data(data_path_normal, data_path_anomaly):
    def split_bucket_parts(path):
        url = urlparse(path)  # [:-1] cuts the \n from the lines
        bucket = "%s://%s" % (url.scheme, url.netloc)

        # [1:] removes leading / to avoid empty string in dir_parts
        dir_parts = os.path.normpath(url.path[1:]).split(os.sep)
        
        return bucket, dir_parts

    def get_pipes(lines):
        splitted_lines = np.array([split_bucket_parts(l)[1] for l in lines])
        noc = list(set(splitted_lines[:, 2]))
        return noc

    def split_train_test(lines, train_pipes):

        # find line without a train class in it
        tts_lines = int(.8 * len(lines))-1
        i = tts_lines
        for l in lines[tts_lines:]:
            if not any(tp in l for tp in train_pipes):
                break
            i += 1

        train_file_paths = lines[:i]
        test_file_paths = lines[i:]
        # print(len(train_file_paths)/len(lines),
        #   len(test_file_paths)/len(lines))
        return train_file_paths, test_file_paths
    # get normal data

    with gfile.GFile(data_path_normal, "r") as f:
        lines = f.readlines()
        lines = lines[0::DATA_SET_FRAC]
        lines = [l[:-1] for l in lines]  # remove trailing \n char
    bucket = split_bucket_parts(lines[0])[0]
    pipes = get_pipes(lines) # pipe names e.g "1-11111 1-11110"
    nop = len(pipes)  # number of pipes
    
    # 80% of the pipe sections should be used for train and validation, therefor pipe section names are split
    train_val_pipes = pipes[:int(.8 * nop) ] # names of pipes which should be used for training and validation

    normal_train_file_paths, normal_test_file_paths = split_train_test(
        lines, train_val_pipes)
    
    train_pipes = pipes[:int(.8 * nop) ] # names of pipes which should be used for training and validation
    normal_train_file_paths, normal_validation_file_paths = split_train_test(
        lines[:len(normal_train_file_paths)], train_pipes)
    
    train_ratio = len(normal_train_file_paths)/len(lines)
    val_ratio = len(normal_validation_file_paths)/len(lines)
    test_ratio = len(normal_test_file_paths)/len(lines)

    # get anomaly data
    with gfile.GFile(data_path_anomaly, "r") as f:
        lines = f.readlines()
        lines = lines[0::(DATA_SET_FRAC//10)]
        file_paths = [split_bucket_parts(l[:-1])[1][-1] for l in lines]

    decoded_lines = []
    for l in file_paths:
        rest = os.path.join(*base64.b64decode(
            l[:-4]).decode("utf-8").split(os.sep)[-3:])+".jpg"
        new_path = os.path.join(bucket,  "out", rest)
        decoded_lines.append(new_path)

    anomaly_train_file_paths, anomaly_test_file_paths = split_train_test(
        decoded_lines, train_pipes)
    
    print("Splitting data:\n%4d Normal Train Samples\n%4d Normal Validation Samples\n%4d Normal Test Samples\n%4d Anomaly Train Samples\n%4d Anomaly Test Samples" % (
        len(normal_train_file_paths), len(normal_validation_file_paths), len(normal_test_file_paths), len(anomaly_train_file_paths), len(anomaly_test_file_paths)))
    print("Train-Val-Test Ratio: %.3f-%.3f-%.3f"%(train_ratio,val_ratio,test_ratio))
    return normal_train_file_paths, normal_validation_file_paths, normal_test_file_paths, anomaly_train_file_paths, anomaly_test_file_paths

In [76]:
# define train params
target_size = (256, 256, 3)
print(os.path.join(args.datadir, "no_damage.txt"))

train_file_names, validation_file_names, normal_test_file_names, anomaly_train_file_names, anomaly_test_file_names = load_data(os.path.join(args.datadir, "no_damage.txt"),
                                                                                                                                                       os.path.join(args.datadir, "damage_encoded.txt"))
# train_generator, validation_generator, test_generator = preproc_data(
#     input_shape, batch_size, args.datadir)

gs://training-data-images/no_damage.txt
Splitting data:
5810 Normal Train Samples
1232 Normal Validation Samples
1685 Normal Test Samples
 452 Anomaly Train Samples
  99 Anomaly Test Samples
Train-Val-Test Ratio: 0.666-0.141-0.193


In [ ]:
def train_img_generator(file_paths, batch_size, target_size, preproc=True):
    while True:
        inds = (np.random.randint(0, len(file_paths), batch_size))
        imgs = np.array([])
        for i in inds:
            fp = file_paths[i]
            # print(fp)
            if gfile.Exists(fp):
                img = filepath_to_image(fp)
                img = cv2.resize(img, target_size[:-1])
                if preproc:
                    np_img = preproc_img(img, flip_top_bottom=False, hsv=None)
                else:
                    np_img = np.array(img) / 255

                imgs = np.concatenate(
                    (imgs, np.array([np_img]))) if imgs.size > 0 else np.array([np_img])
            yield imgs, imgs

In [ ]:
def img_generator(file_paths, n, target_size):
    # returns generator of n images
    if len(file_paths) < n:
        # prevent an IndexError
        n = len(file_paths)
    for i in range(n):
        fp = file_paths[i]
        if gfile.Exists(fp):
            img = filepath_to_image(fp)
            # img = img.resize(target_size[:-1], Image.BICUBIC)
            img = cv2.resize(img, target_size[:-1])
            yield np.array(img) / 255

In [124]:
def filepath_to_image(fp):
    with gfile.GFile(fp, mode="rb") as f:
        # img = Image.open(f)
        img = cv2.imdecode(np.asarray(bytearray(f.read())), 1)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # convert2rgb
    return img

In [80]:
def rand(a=0, b=1):
    """:returns random value between a and b"""
    return np.random.rand()*(b-a) + a


def preproc_img(image, flip_top_bottom=True, flip_left_right=True, hsv=(.1, 1.5, 1.5)):
    """https://github.com/qqwweee/keras-yolo3/blob/master/yolo3/utils.py"""
    # flip image or not
    if flip_left_right and rand() < .5:
        # image = image.transpose(Image.FLIP_LEFT_RIGHT)
        image = cv2.flip(image, flipCode=0)
    if flip_top_bottom and rand() < .5:
        # image = image.transpose(Image.FLIP_TOP_BOTTOM)
        image = cv2.flip(image, flipCode=1)

    # image brighDtness enhancer
    # if brightness:
        # brightness = ImageEnhance.Brightness(image)

        # image = brightness.enhance(rand(.5, 1.5))
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # brightness and contrast

    image = np.array(image)
    image = np.clip((rand(.5, 1.5) * image + rand(-50, 100)), 0, 255)
    image = image.astype(np.uint8)

    # # increase or decrease contrast
    # if contrast:
    #     contrast = ImageEnhance.Contrast(image)
    #     image = contrast.enhance(rand(.5, 1.5))

    image = image / 255.

    if hsv != None:
        hue, sat, val = hsv
        # distort image
        hue = rand(-hue, hue)
        sat = rand(1, sat) if rand() < .5 else 1/rand(1, sat)
        val = rand(1, val) if rand() < .5 else 1 / rand(1, val)

        x = rgb_to_hsv(image)
        x[..., 0] += hue
        x[..., 0][x[..., 0] > 1] -= 1
        x[..., 0][x[..., 0] < 0] += 1
        x[..., 1] *= sat
        x[..., 2] *= val
        x[x > 1] = 1
        x[x < 0] = 0
        image = hsv_to_rgb(x)
    return image  # numpy array, 0 to 1

In [81]:
def split_ae(autoencoder):
    encoder_layer = autoencoder.get_layer("encoder")
    # this model maps an input to its encoded representation; Big image to small rep
    encoder = Model(
        inputs=autoencoder.get_input_at(0), outputs=encoder_layer.output)

    # create a placeholder for an encoded (ENCODING_DIM-dimensional) input
    encoded_input = Input(shape=encoder_layer.output_shape[1:])

    # getting the middle of the autoencoder
    start = (len(autoencoder.layers))//2
    decoder = autoencoder.layers[-start](encoded_input)
    # stacking the decoder layers
    for i in range(start-1, 0, -1):
        decoder = autoencoder.layers[-i](decoder)

    # create the decoder model; "<": encoded(small) representation to big image
    decoder = Model(encoded_input, decoder)
    return encoder, decoder


In [82]:
def build_conv_ae(filters, input_shape=(256, 256, 3)):
    n = 10
    print(filters)
    if len(filters) != n:
        raise ValueError("%d Filters must be given. Sorry." % n)
    # this is our input placeholder
    input_img = Input(shape=input_shape)
    # layer between input and middle layer
    i = 0
    encode = Conv2D(
        filters[i], (4, 4), strides=(2, 2), padding="same"
    )(input_img)
    encode = LeakyReLU(0.2)(encode)
    i += 1
    encode = Conv2D(
        filters[i], (4, 4), strides=(2, 2),  padding="same"
    )(encode)
    encode = LeakyReLU(0.2)(encode)

    i += 1
    encode = Conv2D(
        filters[i], (3, 3), strides=(1, 1),  padding="same"
    )(encode)
    encode = LeakyReLU(0.2)(encode)
    i += 1
    encode = Conv2D(
        filters[i], (4, 4), strides=(2, 2),  padding="same"
    )(encode)
    encode = LeakyReLU(0.2)(encode)
    i += 1
    encode = Conv2D(
        filters[i], (3, 3), strides=(1, 1),  padding="same"
    )(encode)
    encode = LeakyReLU(0.2)(encode)
    i += 1
    encode = Conv2D(
        filters[i], (4, 4), strides=(2, 2),  padding="same"
    )(encode)
    encode = LeakyReLU(0.2)(encode)
    i += 1
    encode = Conv2D(
        filters[i], (4, 4), strides=(2, 2),  padding="same"
    )(encode)
    encode = LeakyReLU(0.2)(encode)
    i += 1
    encode = Conv2D(
        filters[i], (3, 3), strides=(1, 1),  padding="same"
    )(encode)
    encode = LeakyReLU(0.2)(encode)
    i += 1
    encode = Conv2D(
        filters[i], (4, 4), strides=(2, 2),  padding="same"
    )(encode)
    encode = LeakyReLU(0.2)(encode)
    i += 1
    # "encoded" is the encoded representation of the input, middle layer of the aue
    encoded = Conv2D(
        filters[i], (4, 4), strides=(1, 1),  name="encoder"
    )(encode)

    i -= 1
    # layer between middle and output layer
    decode = Conv2DTranspose(filters[i], (4, 4), strides=(1, 1), activation="relu")(
        encoded
    )
    decode = LeakyReLU(0.2)(decode)
    i -= 1
    decode = Conv2DTranspose(
        filters[i], (4, 4), strides=(2, 2),  padding="same"
    )(decode)
    decode = LeakyReLU(0.2)(decode)
    i -= 1
    decode = Conv2DTranspose(
        filters[i], (3, 3), strides=(1, 1),  padding="same"
    )(decode)
    decode = LeakyReLU(0.2)(decode)
    i -= 1
    decode = Conv2DTranspose(
        filters[i], (4, 4), strides=(2, 2),  padding="same"
    )(decode)
    decode = LeakyReLU(0.2)(decode)
    i -= 1
    decode = Conv2DTranspose(
        filters[i], (4, 4), strides=(2, 2),  padding="same"
    )(decode)
    decode = LeakyReLU(0.2)(decode)
    i -= 1
    decode = Conv2DTranspose(
        filters[i], (4, 4), strides=(2, 2),  padding="same"
    )(decode)
    decode = LeakyReLU(0.2)(decode)
    i -= 1
    decode = Conv2DTranspose(
        filters[i], (3, 3), strides=(1, 1),  padding="same"
    )(decode)
    i -= 1
    decode = LeakyReLU(0.2)(decode)
    decode = Conv2DTranspose(
        filters[i], (4, 4), strides=(2, 2),  padding="same"
    )(decode)
    decode = LeakyReLU(0.2)(decode)
    i -= 1
    decode = Conv2DTranspose(
        filters[i], (4, 4), strides=(2, 2),  padding="same"
    )(decode)
    decode = LeakyReLU(0.2)(decode)
    
    decoded = Conv2D(
        input_shape[-1], (3, 3), activation="sigmoid", padding="same"
    )(decode)

    # this model maps an input to its reconstruction
    autoencoder_single = Model(inputs=input_img, outputs=decoded)
    n_gpus = len(get_available_gpus())
    if n_gpus > 1:
        autoencoder = multi_gpu_model(autoencoder_single, n_gpus)
        print("Autoencoder is trained on %d GPUs" % n_gpus)
    else:
        autoencoder = autoencoder_single
        print("Autoencoder is trained on a single GPU")
    
    encoder, decoder = split_ae(autoencoder_single)

    # build (aka "compile") the model
    autoencoder.compile(optimizer="adam", loss="mse")
    autoencoder_single.compile(optimizer="adam", loss="mse")
    return autoencoder, autoencoder_single, encoder, decoder


In [ ]:
def train_ae(train_file_names, validation_file_names,  args,  target_size=(256, 256, 3)):
    print("Writing logs to %s" % args.logdir)
    train_generator = train_img_generator(
        train_file_names, args.batch_size, target_size)
    validation_generator = train_img_generator(
        validation_file_names, args.batch_size, target_size, preproc=False)

    print("Training Autoencoder for %s feature extraction..." % USE_CASE)
    d = args.ldim
    batch_size = args.batch_size
    filters = args.filters + [d]  # [16, 16, 16, 32, 64, 64, 32, 32, d]

    ae, ae_single, encoder, decoder = build_conv_ae(
        input_shape=target_size, filters=filters)
    # define callbacks for logging and optimized training and ckpt saving

    earlyStopping = EarlyStopping(
        monitor="val_loss", patience=20, verbose=1, mode="min", min_delta=(1/10**5)
    )
    # saves model which was trained on a single cpu since saving the other model threw some kind of error
    checkpoint_format = os.path.join(
        args.ckptdir, 'ep{epoch:04d}-loss{loss:.6f}-val_loss{val_loss:.6f}.h5')

    mcp_save = ModelCheckpointGC(
        checkpoint_format, ae_single, save_best_only=True, verbose=1, monitor="val_loss", mode="min"
    )
    reduce_lr_loss = ReduceLROnPlateau(
        monitor="val_loss", factor=0.2, patience=7, verbose=1, mode="min"
    )

    tb = TensorBoard(
        log_dir=args.logdir, histogram_freq=0, write_graph=True, write_images=True
    )

    f = io.StringIO()
    with redirect_stdout(f):
        ae_single.summary()
    summary = f.getvalue()
    print(summary)

    with gfile.GFile(os.path.join(args.logdir, "train-specs.txt"), mode="w") as f:
        f.write("\n".join(("Filters: {}".format(args.filters),
                           "Batch Size: %d" % batch_size,
                           "Latent Space Dim: %d" % d,
                           "Auto Encoder Network %s" % summary)))

    train_length = len(train_file_names)
    val_length = len(validation_file_names)

    ae.fit_generator(
        generator=train_generator,  # needs to produce data infinitely
        epochs=args.epochs,
        # every element once on average
        steps_per_epoch=train_length//batch_size,
        validation_data=validation_generator,
        validation_steps=val_length//batch_size,
        callbacks=[mcp_save, earlyStopping, reduce_lr_loss, tb]
    )

    return ae, ae_single,encoder, decoder

In [84]:
def load(ckpt_path):
    print("Loading Autoencoder for %s feature extraction from directory %s..." % (
        USE_CASE, ckpt_path))

    url = urlparse(ckpt_path)  # [:-1] cuts the \n from the lines
    bucket = "%s://%s" % (url.scheme, url.netloc)

    # [1:] removes leading / to avoid empty string in dir_parts
    dir_parts = os.path.normpath(url.path[1:]).split(os.sep)

    fp = os.path.join("tmp", dir_parts[-1])
    if not os.path.isdir("tmp"):
        os.mkdir("tmp")

    if not os.path.isfile(fp):
        subprocess.call(["gsutil", "-m", "cp", ckpt_path, "tmp"])
    ae = load_model(fp)

    # this needs to be compiled since the untrained, single-GPU model is saved instead of the multi-GPU model
    ae.compile(optimizer="adam", loss="mse")
    encoder, decoder = split_ae(ae)
    return ae, encoder, decoder

In [102]:
ae_multi,ae, _, _ = train_ae(train_file_names, validation_file_names,  args,  target_size=target_size)
# ae, _, _ = load(
#        "gs://e4u-anomaly-detection/ckpts/AD_spm_20190813_144154/ep0057-loss0.009716-val_loss0.015439.h5")

Writing logs to gs://e4u-anomaly-detection/logs/DEBUG_AD
Training Autoencoder for spm feature extraction...
[32, 32, 32, 64, 64, 128, 256, 128, 64, 256]
Autoencoder is trained on 2 GPUs
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 128, 128, 32)      1568      
_________________________________________________________________
leaky_re_lu_73 (LeakyReLU)   (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 64, 64, 32)        16416     
_________________________________________________________________
leaky_re_lu_74 (LeakyReLU)   (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_43 (Conv2D)    

In [103]:
def predict_from_generator(model, gen, batch_size):
    predictions = []
    for _ in gen:
        batch = []
        for _ in range(batch_size):
            try:
                batch.append(next(gen))
            except StopIteration:
                pass
        pred = model.predict(np.array(batch))
        # pred[0], because prediction happens on an array, hence result has also length one
        predictions.extend(pred)
        print(len(predictions), "predictions made")
    return predictions

In [104]:
def evaluate(ae, loss_boundary, oc_svm,  X_normal_test, X_anomaly_test, img_dir, target_size):
    # load the data for evaluation purposes
    print("Evaluating feature extractor...")
    # eval_train = model.evaluate_generator(
    #     train_img_generator(X_normal_train, 8), steps=len(X_normal_train))
    batch_size = 64

    eval_test = ae.evaluate_generator(
        train_img_generator(X_normal_test, batch_size, target_size, preproc=False), steps=len(X_normal_test)//batch_size)
    # print("Feature extractor train loss: %f" % eval_train)

    print("Feature extractor  test loss: %f" % eval_test)

    decoded_samples_normal = predict_from_generator(
        ae, img_generator(X_normal_test, 8, target_size),batch_size)

    plot_samples(
        img_generator(X_normal_test, 8, target_size),
        decoded_samples_normal,
        plot_mvtec,
        os.path.join(img_dir, "rec-normals.png"),
    )

    decoded_samples_anomaly = predict_from_generator(
        ae, img_generator(X_anomaly_test, batch_size, target_size),batch_size)

    plot_samples(
        img_generator(X_anomaly_test, batch_size, target_size),
        decoded_samples_anomaly,
        plot_mvtec,
        os.path.join(img_dir, "rec-anomalies.png"),
    )

    metrics = {}

    print("Evaluating the loss based approach")
    metrics["loss"] = eval_loss(model=ae, X_normal=X_normal_test,
                                X_anomaly=X_anomaly_test, loss_boundary=loss_boundary, img_dir=img_dir, target_size=target_size)
    print("Evaluating the OC SVM")

    encoder, _ = split_ae(ae)
    metrics["svm"] = eval_svm(encoder, oc_svm,
                              X_normal_test, X_anomaly_test, img_dir, target_size, batch_size)
    print(metrics)
    return metrics

In [105]:
def get_metrics(TP, TN, FP, FN):
    try:
        precision = TP / (TP + FP)
    except ZeroDivisionError:
        precision = 0
    try:
        recall = TP / (TP + FN)
    except ZeroDivisionError:
        recall = 0

    accuracy = (TN + TP) / (TP + TN + FN + FP)
    TPR = recall
    try:
        TNR = TN / (TN + FP)
    except ZeroDivisionError:
        TNR = 0
    
    FNR = 1 - TPR
    FPR = 1 - TNR
    
    try:
        F_1 = 2 * (precision * recall) / (precision + recall)
    except ZeroDivisionError:
        F_1 = 0
    try:
        F_2 = 5 * (precision * recall) / (4*precision + recall)
    except ZeroDivisionError:
        F_2 = 0

    try:
        # between -1 and 1
        MCC = (TP*TN - FP*FN)/(np.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)))
    except ZeroDivisionError:
        MCC = 0
    current_metrics = {"precision": precision,
                       "accuracy": accuracy,
                       "recall": recall,
                       "F1_score": F_1,
                       "F2_score": F_2,
                       "TPR": TPR,
                       "TNR": TNR,
                       "FNR": FNR,
                       "FPR": FPR,
                       "MCC": MCC
                       }
    return current_metrics

In [106]:
def ROC_curve(metrics, file_path):
    # TODO: fix plot saving
    fig = plt.figure()
    plt.plot([m["FPR"] for m in metrics],
             [m["TPR"] for m in metrics])
    plt.grid()
    plt.xlabel("FALSE POSITIVE RATE")
    plt.ylabel("TRUE POSITIVE RATE")
    savefig(fig, file_path)

    plt.clf()

In [107]:
def loss_per_img(img, rec_img):
    # mean squared error
    return np.sum(np.power(rec_img - img, 2))  # / (np.prod(img.shape) - 1)


In [108]:
def get_losses(model, X_normal, X_anomaly, target_size):
    batch_size = 64
    print("Computing losses for normal samples.")
    samples_normal = img_generator(
        X_normal, len(X_normal), target_size
    )
    print("Reconstructing normal images.")
    decoded_samples_normal = predict_from_generator(model, samples_normal,batch_size=batch_size)
    print("Computing normal losses per image")
    normal_losses = np.array([loss_per_img(i, ri)for i, ri in zip(
        img_generator(X_normal, len(X_normal), target_size), decoded_samples_normal)])
    
    print("Computing losses for anomalous samples.")
    print("Reconstructing anomalous images.")
    decoded_samples_anomaly = predict_from_generator(model, img_generator(
        X_anomaly, len(X_anomaly), target_size
    ),batch_size=batch_size)
    
    print("Computing anomaly losses per image")
    anomaly_losses = np.array([loss_per_img(i, ri)for i, ri in zip(
        img_generator(X_anomaly, len(X_anomaly), target_size), decoded_samples_anomaly)])
    return normal_losses, anomaly_losses

In [109]:
def eval_loss(model, X_normal, X_anomaly, loss_boundary, img_dir, target_size):
    """anomaly detection based on the loss"""

    # compute losses on the test set
    normal_losses, anomaly_losses = get_losses(model,
                                               X_normal, X_anomaly, target_size)
    bins = 10
    # loss distribution over the normal dataset
    fig = plt.figure()
    label = "Distribution of normal loss values"
    plot_hist(normal_losses, relative=True,
              color="g", bins=bins, label=label)

    label = "Distribution of normal loss values"
    # loss distribution over the anomaly dataset
    plot_hist(anomaly_losses, relative=True,
              color="r", bins=bins, label=label)

    savefig(fig, os.path.join(img_dir, "loss-dist.png"))
    plt.clf()

    # ground truth: negatives = anomaly
    TP = anomaly_losses[anomaly_losses >= loss_boundary]
    FN = anomaly_losses[anomaly_losses < loss_boundary]

    # ground truth: positives = normality
    TN = normal_losses[normal_losses < loss_boundary]
    FP = normal_losses[normal_losses >= loss_boundary]

    # ROC_curve(metrics, os.path.join(img_dir, "loss-ROC.png"))
    return get_metrics(len(TP), len(TN), len(FP), len(FN))

In [110]:
def eval_svm(encoder, oc_svm,   X_normal, X_anomaly, img_dir, target_size, batch_size):

    print("Encoding normal test images to latent space...")
    encoded_normal_imgs_test = encoder.predict_generator(
        train_img_generator(X_normal, batch_size, target_size, preproc=False), steps=len(X_normal)/batch_size)
    print("Encoding anomaly test images to latent space...")
    encoded_anomaly_imgs_test = encoder.predict_generator(
        train_img_generator(X_anomaly, batch_size, target_size, preproc=False), steps=len(X_anomaly)/batch_size)

    # reshape to suitable shape for OCC
    encoded_normal_imgs_test = encoded_normal_imgs_test.reshape(
        -1, np.prod(encoded_normal_imgs_test.shape[1:]))
    encoded_anomaly_imgs_test = encoded_anomaly_imgs_test.reshape(
        - 1, np.prod(encoded_anomaly_imgs_test.shape[1:]))

    score_normal = oc_svm.predict(encoded_normal_imgs_test)
    score_anomaly = oc_svm.predict(encoded_anomaly_imgs_test)

    bins = 10
    # loss distribution over the normal dataset
    fig = plt.figure()
    label = "Distribution of the normal svm score values"
    plot_hist(oc_svm.decision_function(encoded_normal_imgs_test), relative=True,
              color="g", bins=bins, label=label)

    label = "Distribution of normal svm score values"
    # loss distribution over the anomaly dataset
    plot_hist(oc_svm.decision_function(encoded_anomaly_imgs_test), relative=True,
              color="r", bins=bins, label=label)

    savefig(fig, os.path.join(img_dir, "svm-score-dist.png"))
    # clear figure
    plt.clf()

    # Normalities are negatives (no finding)

    # ground truth: anomaly
    TP = len(score_anomaly[score_anomaly == -1])
    FN = len(score_anomaly[score_anomaly == 1])

    # ground truth: Normality
    TN = len(score_normal[score_normal == 1])
    FP = len(score_normal[score_normal == -1])

    # ROC_curve(metrics, os.path.join(img_dir, "svm-ROC.png"))
    return get_metrics(TP, TN, FP, FN)

In [111]:
def train_loss(model, normal_paths, anomaly_paths, target_size):
    print("Training the loss.")
    normal_losses, anomaly_losses = get_losses(
        model, normal_paths, anomaly_paths, target_size)
    # generate new samples of normals and anomalies to determine the best boundary on the train set
    # samples_normal = np.array(
    #    list(img_generator(normal_paths, len(normal_paths), target_size)))  
    # samples_normal_generator = img_generator(normal_paths, len(normal_paths), target_size)
    # samples_anomaly = np.array(
    #   list(img_generator(anomaly_paths, len(anomaly_paths), target_size)))
    
    best_m = -1
    best_boundary = 0
    print("Determining best loss boundary for train set.")
    decisive_metric = "MCC"
    for loss_boundary in normal_losses:
        # ground truth: positives = normality
        TP = normal_losses[normal_losses < loss_boundary]
        FN = normal_losses[normal_losses >= loss_boundary]

        # ground truth: negatives = anomaly
        TN = anomaly_losses[anomaly_losses >= loss_boundary]
        FP = anomaly_losses[anomaly_losses < loss_boundary]
        
        current_metrics = get_metrics(
            len(TP), len(TN), len(FP), len(FN))

        # less or equal since we want the biggest TP_rate (i)
        if current_metrics[decisive_metric] >= best_m:
            best_m = current_metrics[decisive_metric]
            best_boundary = loss_boundary
            print("New best metric:",best_m,"Boundary Loss = ", best_boundary)

    return best_boundary

In [112]:
anomaly_train_file_names[:5]

['gs://training-data-images/out/P04033/1-37029 1-37028/back_1m50.jpg',
 'gs://training-data-images/out/P04033/1-37029 1-37028/front_1m50.jpg',
 'gs://training-data-images/out/P04033/1-37031 1-37032/front_19m90.jpg',
 'gs://training-data-images/out/P04033/1-37031 1-37032/front_22m85.jpg',
 'gs://training-data-images/out/P04033/1-37031 1-37032/front_31m55.jpg']

In [113]:
loss_boundary = train_loss(
    ae, train_file_names, anomaly_train_file_names, target_size)
loss_boundary

Training the loss.
Computing losses for normal samples.
Reconstructing normal images.
64 predictions made
128 predictions made
192 predictions made
256 predictions made
320 predictions made
384 predictions made
448 predictions made
512 predictions made
576 predictions made
640 predictions made
704 predictions made
768 predictions made
832 predictions made
896 predictions made
960 predictions made
1024 predictions made
1088 predictions made
1152 predictions made
1216 predictions made
1280 predictions made
1344 predictions made
1408 predictions made
1472 predictions made
1536 predictions made
1600 predictions made
1664 predictions made
1728 predictions made
1792 predictions made
1856 predictions made
1920 predictions made
1984 predictions made
2048 predictions made
2112 predictions made
2176 predictions made
2240 predictions made
2304 predictions made
2368 predictions made
2432 predictions made
2496 predictions made
2560 predictions made
2624 predictions made
2688 predictions made
2752 p

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in double_scalars


1798.8555177283058

In [114]:
def train_svm(model, normal_file_names,  target_size, batch_size):
    print("Training the OC SVM")
    print("Encoding train images to latent space...")
    # encode normal instances to latent space
    encoded_normal_imgs_train = model.predict_generator(
        train_img_generator(normal_file_names, batch_size, target_size, preproc=False), steps=len(normal_file_names) / batch_size)  # used later for One Class Classification

    encoded_normal_imgs_train = encoded_normal_imgs_train.reshape(
        - 1, np.prod(encoded_normal_imgs_train.shape[1:]))

    # # encode anomalies to latent space
    # encoded_anomaly_imgs_train = model.predict_generator(
    #     train_img_generator(anomaly_file_names, batch_size, target_size, preproc=False), steps=len(anomaly_file_names)/batch_size)  # used later for One Class Classification
    # encoded_anomaly_imgs_train = encoded_anomaly_imgs_train.reshape(
    #     - 1, np.prod(encoded_anomaly_imgs_train.shape[1:]))

    print("Fitting the OC SVM")
    clf = svm.OneClassSVM()
    # TODO: evaluate different nu values
    clf.fit(encoded_normal_imgs_train)

    # # computes signed distance to the hyperplane
    # score_normal = clf.predict(encoded_normal_imgs_train)
    # score_anomaly = clf.predict(encoded_anomaly_imgs_train)

    # # ground truth: anomaly
    # TP = len(score_anomaly[score_anomaly == -1])
    # FN = len(score_anomaly[score_anomaly == 1])

    # # ground truth: Normality
    # TN = len(score_normal[score_normal == 1])
    # FP = len(score_normal[score_normal == -1])

    # current_metrics = get_metrics(TP, TN, FP, FN)
    # print("SVM train results:", current_metrics)

    return clf

In [115]:
encoder = split_ae(ae)[0]
oc_svm = train_svm(encoder, train_file_names,
                       target_size, batch_size=args.batch_size)

Training the OC SVM
Encoding train images to latent space...
Fitting the OC SVM


/usr/local/lib/python3.5/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [127]:
# returns metrics dictionary
metrics = evaluate(ae, loss_boundary, oc_svm, normal_test_file_names,
                       anomaly_test_file_names, args.imgdir, target_size)
with gfile.GFile(os.path.join(args.evaldir, "metrics.json"), "w") as f:
    json.dump(metrics, f)

Evaluating feature extractor...
Feature extractor  test loss: 0.012908
7 predictions made
63 predictions made
Evaluating the loss based approach
Computing losses for normal samples.
Reconstructing normal images.
64 predictions made
128 predictions made
192 predictions made
256 predictions made
320 predictions made
384 predictions made
448 predictions made
512 predictions made
576 predictions made
640 predictions made
704 predictions made
768 predictions made
832 predictions made
896 predictions made
960 predictions made
1024 predictions made
1088 predictions made
1152 predictions made
1216 predictions made
1280 predictions made
1344 predictions made
1408 predictions made
1472 predictions made
1536 predictions made
1600 predictions made
1659 predictions made
Computing normal losses per image
Computing losses for anomalous samples.
Reconstructing anomalous images.
64 predictions made
97 predictions made
Computing anomaly losses per image
Evaluating the OC SVM
Encoding normal test images 

<Figure size 1152x288 with 0 Axes>

<Figure size 1152x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

In [129]:
!gsutil -m cp -r gs://e4u-anomaly-detection/*/DEBUG_AD .

Copying gs://e4u-anomaly-detection/ckpts/DEBUG_AD/ep0000-loss0.056312-val_loss0.018705.h5...
Copying gs://e4u-anomaly-detection/ckpts/DEBUG_AD/ep0000-loss0.061939-val_loss0.050991.h5...
Copying gs://e4u-anomaly-detection/ckpts/DEBUG_AD/ep0000-loss0.064818-val_loss0.020767.h5...
Copying gs://e4u-anomaly-detection/ckpts/DEBUG_AD/ep0000-loss0.094021-val_loss0.092254.h5...
Copying gs://e4u-anomaly-detection/ckpts/DEBUG_AD/ep0001-loss0.024045-val_loss0.012379.h5...
Copying gs://e4u-anomaly-detection/ckpts/DEBUG_AD/ep0001-loss0.027981-val_loss0.015439.h5...
Copying gs://e4u-anomaly-detection/ckpts/DEBUG_AD/ep0001-loss0.028694-val_loss0.038799.h5...
Copying gs://e4u-anomaly-detection/ckpts/DEBUG_AD/ep0001-loss0.039759-val_loss0.049159.h5...
Copying gs://e4u-anomaly-detection/ckpts/DEBUG_AD/ep0002-loss0.020666-val_loss0.013791.h5...
Copying gs://e4u-anomaly-detection/ckpts/DEBUG_AD/ep0002-loss0.020791-val_loss0.024569.h5...
Copying gs://e4u-anomaly-detection/ckpts/DEBUG_AD/ep0003-loss0.012724-